In [1]:
# Modules, functions -- 

import numpy as np
import pandas as pd
from pandas import DataFrame, Series

import multiprocessing

import matplotlib.pyplot as plt
import bokeh
import bokeh.io
from bokeh.plotting import figure
from bokeh.io import output_notebook, show

# init_notebook_mode()

import seaborn as sns

import re
import math
import copy

from collections import defaultdict
import csv
import itertools
import datetime 
from datetime import datetime
import time
import dateutil.parser
import pickle
import random

import gc
import zipfile
import sys, getopt
import os

from IPython.core.interactiveshell import InteractiveShell
from io import StringIO

# import dask.dataframe as dd
#from chest import Chest

InteractiveShell.ast_node_interactivity = "all"
# InteractiveShell.ast_node_interactivity = "last"

# Magic function to make matplotlib inline
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}

# Set up Bokeh for inline viewing
bokeh.io.output_notebook()

# import dask.dataframe as ddf
# import dask.array as da

pd.set_option('max_columns', 500)
pd.set_option('max_rows', 1700)

import scipy

import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.tsatools import detrend

import datetime as dt

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

Loading BokehJS ...

In [2]:
# preds = pd.read_excel('FW20_forecasts_incl_cl.xlsx')

rbkFW20_0 = pd.read_excel('diff_article_numbers.xlsx', skiprows=8)

ref_dat0 = pd.read_csv('data/article_reference_data.csv', low_memory = False, error_bad_lines = False, 
    usecols = ['article_no', 'model_no', 'art_desc', 'brand_desc', 'colorway_long_descr',
               'primary_color', 'secondary_color', 'tertiary_color', 'quarternary_color'])

In [45]:
rbkFW20 = rbkFW20_0.copy()
ref_dat = ref_dat0.copy()

In [46]:
# Just the rbk w/ stealth model
rbkFW20 = rbkFW20[['Art #', 'Model ID', 'model ID FW19', 'Colorway']]
rbkFW20 = rbkFW20[~rbkFW20['model ID FW19'].isna()] 
rbkFW20['Art #'] = [str(x).zfill(6) for x in rbkFW20['Art #']]

In [47]:
seasons = pd.read_csv('data/EU_seasons.csv', low_memory = False, error_bad_lines = False, sep = ",").sort_values(['article_number', 'brand', 'season']) # 26 Aug
rbkFW19 = seasons[(seasons.season == 'FW19') & (seasons.brand == 'REEBOK')]

rbkFW19 = pd.merge(rbkFW19, ref_dat, left_on = 'article_number', right_on= 'article_no', how = 'left') # Add model numbers

rbkFW19 = rbkFW19.dropna(subset = ['model_no']) # remove missing model number rows


rbkFW19 = rbkFW19[rbkFW19.model_no.isin(rbkFW20['model ID FW19'])] # only models that are stealth candidates 
rbkFW19 = (rbkFW19[['article_number', 'season', 'model_no', 'colorway_long_descr', 'primary_color', 'secondary_color', 'tertiary_color', 'quarternary_color']].
           rename(columns = {'model_no': 'model_no_FW19', 'article_number': 'article_number_FW19'})) # color data



In [19]:
rbkFW20['Art #'].isin(rbkFW19.article_number).sum() # 99 carryovers among the 417 articles

99

In [48]:
rbkFW20 = (pd.merge(rbkFW20, ref_dat0[['article_no', 'model_no', 'colorway_long_descr', 'primary_color', 'secondary_color']], 
                    left_on=['Art #', 'Model ID'], right_on=['article_no', 'model_no'], how = 'left').
           drop(['Art #', 'Model ID'], axis = 1).
           rename(columns = {'model ID FW19': 'model_no_FW19', 'article_no': 'article_number'}).
           set_index('article_number')) # add color

rbkFW20.shape # (417, 6)
rbkFW20 = rbkFW20[~rbkFW20.index.isin(rbkFW19.article_number_FW19)] # 318 remain
rbkFW20.shape # (318, 6)

(417, 6)

(318, 6)

In [49]:
rbkFW20.head() # FW20 w/ reference
rbkFW19.head() # FW19 w/ reference

# rbkFW20.index.isin(rbkFW19.article_number).sum() # 99 carryovers among the 417 articles

,model_no_FW19,Colorway,model_no,colorway_long_descr,primary_color,secondary_color
article_number,,,,,,
050190,MU793,BLACK-1,MU793,BLACK-1,NaN,NaN
AR0455,AVL59,INT-WHITE/SHEER GREY,AVL59,INT-WHITE/SHEER GREY,NaN,NaN
DV5834,EGY17,COLD GREY/CHALK/SOFT CAME,EGY17,COLD GREY/CHALK/SOFT CAME,NaN,NaN
EH0861,AWL82,white/cold grey 2/collegiate royal,IB867,WHITE/CDGRY2/CROYAL,White,Grey
EH0862,AWL82,white/collegiate navy/collegiate royal,IB867,WHITE/CONAVY/CROYAL,White,Blue


,article_number_FW19,season,model_no_FW19,colorway_long_descr,primary_color,secondary_color,tertiary_color,quarternary_color
2,002214,FW19,OC079,INT-WHITE/LT. GREY,NaN,NaN,NaN,NaN
4,002232,FW19,OC078,INT-WHITE,NaN,NaN,NaN,NaN
5,002240,FW19,FF125,INT-BLACK,NaN,NaN,NaN,NaN
6,002267,FW19,OC079,INT-BLACK,NaN,NaN,NaN,NaN
8,002431,FW19,FF125,INT-WHITE/SILVER,NaN,NaN,NaN,NaN


In [50]:
rbkFW20 = rbkFW20[~rbkFW20.index.isin(rbkFW19.article_number_FW19)] # 318 remain

In [57]:
# Combine FW20 articles with FW19 potential stealths
rbk1920 = pd.merge(rbkFW20.reset_index(), rbkFW19, how = 'left', on = 'model_no_FW19').drop(['Colorway', 'tertiary_color', 'quarternary_color', 'season'], axis = 1)

# x = FW20
# y = FW19

,article_number,model_no_FW19,model_no,colorway_long_descr_x,primary_color_x,secondary_color_x,article_number_FW19,colorway_long_descr_y,primary_color_y,secondary_color_y
0,050190,MU793,MU793,BLACK-1,NaN,NaN,050192,WHITE-1,Black,Grey
1,AR0455,AVL59,AVL59,INT-WHITE/SHEER GREY,NaN,NaN,AR0454,INT-BLACK/CHARCOAL,NaN,NaN


In [60]:
rbk1920 = rbk1920.rename(columns = {
                                     'colorway_long_descr_x': 'colorway_long_FW20',
                                     'primary_color_x':       'primary_FW20', 
                                     'secondary_color_x':     'secondary_FW20', 
                                     
                                     'colorway_long_descr_y': 'colorway_long_FW19', 
                                     'primary_color_y':       'primary_FW19',
                                     'secondary_color_y':     'secondary_FW19'})

rbk1920 = rbk1920[['article_number', 'model_no', 'colorway_long_FW20',
       'primary_FW20', 'secondary_FW20', 'article_number_FW19', 'model_no_FW19', 
       'colorway_long_FW19', 'primary_FW19', 'secondary_FW19']]

rbk1920 = rbk1920.dropna(subset = ['colorway_long_FW19'])


Index(['article_number', 'model_no_FW19', 'model_no', 'colorway_long_FW20',
       'primary_FW20', 'secondary_FW20', 'article_number_FW19',
       'colorway_long_FW19', 'primary_FW19', 'secondary_FW19'],
      dtype='object')

In [65]:
rbk1920['FW19_color'] = [x.split('/') for x in rbk1920.colorway_long_FW19]
rbk1920['FW20_color'] = [x.split('/')  for x in rbk1920.colorway_long_FW20]


In [70]:
# Matching criteria
rbk1920['c1'] = rbk1920.FW19_color == rbk1920.FW20_color 
rbk1920['c2'] = rbk1920.primary_FW19 == rbk1920.primary_FW20
rbk1920['c3'] = [x[0] == y[0] for x, y in zip(rbk1920.FW19_color, rbk1920.FW20_color)]


In [84]:
rbk1920.article_number_FW19.unique().size
rbk1920.article_number.unique().size

rbk1920.shape

347

315

(1217, 15)

In [ ]:
stealths = rbk1920[c1]
stealths2 = rbk1920[c2 & ~rbk1920.article_number_FW20.isin(stealths.article_number_FW20)]
stealths3 = rbk1920[c3 & ~rbk1920.article_number_FW20.isin(stealths.article_number_FW20) & ~rbk1920.article_number_FW20.isin(stealths2.article_number_FW20)]

In [ ]:
stealths.article_number_FW20.unique().size
stealths.shape

stealths2.article_number_FW20.unique().size
stealths2.shape

stealths3.article_number_FW20.unique().size
stealths3.shape

# ADD COLUMN FOR CRITERIA NUMBER --- for choosing when duplicates

In [ ]:
stealths.columns

In [ ]:
stealths2[stealths2.duplicated(subset = 'article_number_FW20', keep = False)][['article_number_FW20', 'model_FW20', 'FW20_color', 'primary_FW20', 
                                                                               'secondary_FW20', 'article_number_FW19', 'model_FW19_stealth', 
                                                                               'primary_FW19', 'secondary_FW19',
                                                                               'FW19_color']]

In [ ]:
stealths = pd.concat([stealths, stealths2, stealths3])

In [85]:
rbk1920.to_excel('data/rbk_1920.xlsx')

In [ ]:
stealths[stealths.article_number_FW20 == 'FU2373'] #duplicated(subset = ['article_number_FW20'], keep = False)]

In [ ]:
stealths.article_number_FW20.unique().size